In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append('%s/../' % os.getcwd())

import datetime
import pandas as pd
import numpy as np
import getpass

In [2]:
#importa as fórmulas do arquivo aluguel_offshore.py - note que este arquivo deverá estar na mesma pasta deste notebook
from aluguel_offshore import *
from confirmacoes.aux_emails import email_sender

In [3]:
#pasta em que estão os arquivos no seu computador.
pasta = r'C:\Users\correigu\OneDrive - Banco BTG Pactual S.A\Desktop\aqruivos_offshore'
#pasta = r'C:\Users\correigu\OneDrive - Banco BTG Pactual S.A\Desktop\aqruivos_offshore\janfev'

In [4]:
data_inicio = pd.to_datetime('2018-01-01')
data_final = pd.to_datetime('2018-08-30')

In [5]:
def pega_data_file(file):
    lista_erros = []
    try:
        try:
            data = pd.to_datetime(file[-12:-4], format='%Y%m%d')
        except ValueError:
            try:
                data = pd.to_datetime(file[-14:-4], format='%Y-%m-%d')
            except ValueError:
                data = pd.to_datetime(file[-13:-5], format='%Y%m%d')    
    except ValueError:
        print ('O arquivo %s não possui data registrada' %(file))
        data = 0
    return data

In [6]:
lista_files = []
for file in os.listdir(pasta):
    data = pega_data_file(file)
    
    try:
        if (data>=data_inicio)&(data<=data_final):
            lista_files.append(pasta+'\\'+file)
    except TypeError:
        print ('O arquivo %s não pôde ser comparado com os demais' %(file))

O arquivo Ajustes não possui data registrada
O arquivo Ajustes não pôde ser comparado com os demais
O arquivo old_v2 não possui data registrada
O arquivo old_v2 não pôde ser comparado com os demais
O arquivo tickers_diffs.xlsx não possui data registrada
O arquivo tickers_diffs.xlsx não pôde ser comparado com os demais


In [7]:
%%time
ubs_count=0
morgan_count=0
pras_count=0
bdwm_count=0
intc_count=0     
for file in lista_files:
    if 'Ajuste' in file:
        data = file[-14:-4]
        try:
            ubs
        except:
            ubs = pd.read_csv(file)
            ubs['date']=data
        else:
            ubs_1 = pd.read_csv(file)
            ubs_1['date']=data
            ubs = pd.concat([ubs,ubs_1], axis=0)
        ubs_count+=1

    elif 'PROP+ASSET' in file:
        try:
            pras
        except:
            pras = pd.read_excel(file)
            pras['date'] = pras.columns[7]
            pras = pras.rename(columns={pras.columns[7]:'shares'})
        else:
            pras_1 = pd.read_excel(file)
            pras_1['date'] = pras_1.columns[7]
            pras_1 = pras_1.rename(columns={pras_1.columns[7]:'shares'})
            pras = pd.concat([pras,pras_1], axis=0)

        pras_count+=1

    elif 'Aluguel INT C' in file:
        try:
            intc
        except:
            intc = pd.read_excel(file)
            intc['date'] = intc.columns[7]
            intc = intc.rename(columns={intc.columns[7]:'shares'})
        else:
            intc_1 = pd.read_excel(file)
            intc_1['date'] = intc_1.columns[7]
            intc_1 = intc_1.rename(columns={intc_1.columns[7]:'shares'})
            intc = pd.concat([intc,intc_1], axis=0)

        intc_count+=1

    elif 'Clients BD+WM' in file:
        try:
            bdwm
        except:
            bdwm = pd.read_excel(file)
            bdwm['date'] = bdwm.columns[31]
            bdwm = bdwm.rename(columns={bdwm.columns[31]:'shares'})
        else:
            bdwm_1 = pd.read_excel(file)
            bdwm_1['date'] = bdwm_1.columns[31]
            bdwm_1 = bdwm_1.rename(columns={bdwm_1.columns[31]:'shares'})
            bdwm = pd.concat([bdwm,bdwm_1], axis=0)

        bdwm_count+=1

    elif ('038CDBD08_IN150DX' in file)|('038CDB895_IN150DX' in file):
        try:
            morgan
        except:
            morgan = pd.read_csv(file, skipfooter=1, engine='python')
        else:
            morgan_1 = pd.read_csv(file, skipfooter=1, engine='python')
            morgan = pd.concat([morgan, morgan_1], axis = 0)

        morgan_count+=1


ticker_diffs = pd.read_excel(pasta+'\\'+ 'tickers_diffs.xlsx')

print(str(ubs_count) + ' aqrivos do ubs importados')
print(str(morgan_count) + ' aqrivos da morgan importados')
print(str(pras_count) + ' aqrivos do Calypso PROP+ASSET importados')
print(str(bdwm_count) + ' aqrivos do Calypso BD+WM importados')
print(str(intc_count) + ' aqrivos do Calypso INTC importados')

188 aqrivos do ubs importados
342 aqrivos da morgan importados
170 aqrivos do Calypso PROP+ASSET importados
168 aqrivos do Calypso BD+WM importados
180 aqrivos do Calypso INTC importados
Wall time: 5min 46s


# Ajustes base

In [8]:
ubs['date'] = pd.to_datetime(ubs['date'], format='%Y-%m-%d')
ubs['date'] = np.where(ubs['date'].dt.weekday==6, ubs['date']-datetime.timedelta(2), ubs['date'])
ubs['pb'] = 'ubs'
ubs['month_number'] = ubs['date'].dt.month
ubs['year'] = ubs['date'].dt.year
ubs['month'] = ubs['month_number'].map(months_dict)
ubs['cusip'] = np.nan

In [13]:
ubs = ubs[['Reference Account Id' ,'pb', 'date', 'Ticker', 'ISIN', 'cusip', 'SEDOL', 'Quantity',
           'Daily Accrual', 'Billing CCY', 'month', 'year']].drop_duplicates()
ubs.columns = ['conta', 'pb', 'data_ref', 'ticker', 'isin', 'cusip', 'sedol', 'qt', 'accrual', 'currency', 'month', 'year']
ubs = ubs.drop_duplicates(subset=['conta', 'data_ref', 'ticker', 'qt', 'currency'])
ubs = ubs[ubs['conta'].isnull()==False]

In [10]:
morgan['date'] = pd.to_datetime(morgan['Value Date'], errors='coerce', format = '%m/%d/%Y')
morgan['pb'] = 'ms'
morgan['month_number'] = morgan['date'].dt.month
morgan['year'] = morgan['date'].dt.year
morgan['month'] = morgan['month_number'].map(months_dict)

In [14]:
morgan = morgan[['Account', 'pb', 'date', 'Symbol', 'ISIN', 'Cusip', 'Sedol', 'Shares', 'Net Borrow Cost', 'Currency', 'month', 'year']] 
morgan.columns = ['conta', 'pb', 'data_ref', 'ticker', 'isin', 'cusip', 'sedol', 'qt', 'accrual', 'currency', 'month', 'year']  
nparray = np.where(morgan['data_ref'].dt.weekday==6, morgan['data_ref']-datetime.timedelta(2),morgan['data_ref'])
morgan = morgan.drop('data_ref', axis=1)
morgan['data_ref'] = nparray
morgan = morgan.drop_duplicates().reset_index(drop=True) 
morgan = morgan[morgan['conta'].isnull()==False]

In [15]:
pb = pd.concat([morgan, ubs], axis=0).reset_index(drop=True)
pb.qt = (pb.qt.replace( '[)]','', regex=True )
           .replace( '[(]','-',   regex=True ).astype(float))    
pb.accrual = (pb.accrual.replace( '[)]','', regex=True )
           .replace( '[(]','-',   regex=True ).astype(float))
pb['qt'] = -abs(pb['qt'])
pb = pb[pb.data_ref.dt.weekday<5]
#Como estamos pegando somente os dias úteis, multiplica o valor de sexta por 3 para levar em consideração dias corridos
nparray = np.where(pb.data_ref.dt.weekday==4, pb['accrual']*3, pb['accrual'])
pb['accrual'] = nparray
contas =  pb['conta'].astype(str).copy()
pb['conta'] = contas
#Aplica o dicionário de contas que também está no aluguel_offshore.py
pb['acc'] = pb['conta'].apply(lambda x: dict_contas[x])
pb = pb[['conta', 'pb', 'data_ref', 'ticker', 'isin', 'cusip','sedol','qt', 'accrual','currency','month','year']]

In [16]:
%%time
pras['tipo'] = 'ASSET'
bdwm['tipo'] = 'WM'
intc['tipo'] = 'INT C'
calypso = pd.concat([pras, bdwm, intc], axis=0).reset_index(drop=True)    
calypso['date'] = pd.to_datetime(calypso['date'])
calypso['month_number'] = calypso['date'].dt.month
calypso['year'] = calypso['date'].dt.year
calypso['month'] = calypso['month_number'].map(months_dict)
calypso = calypso[['Account Name','tipo', 'Agent','date', 'shares', 'Book', 'PRODUCT_CODE.TICKER', 'PRODUCT_CODE.ISIN', 
                   'PRODUCT_CODE.CUSIP', 'Underlying.Product Code.SEDOL', 'Product Currency', 'month', 'year']].reset_index(drop=True)
calypso.columns = ['conta_pb','conta', 'agent', 'data_ref', 'quantidade', 'book', 'ticker', 'isin', 'cusip', 'sedol',
                   'currency', 'month', 'year']

Wall time: 1min 28s


In [17]:
calypso['quantidade'] = calypso['quantidade'].replace(',','', regex=True)
calypso['quantidade'] = calypso['quantidade'].astype(float)

In [18]:
calypso.conta_pb = calypso.conta_pb.astype(str)
calypso = calypso[calypso.data_ref.dt.weekday<5]
calypso_1 = calypso.copy()
calypso_1['acc'] = calypso_1['conta_pb'].map(dict_contas)
#Pega somente os arquivos do calypso que se referem às contas da asset
calypso = calypso[calypso.conta_pb.isin(dict_contas.keys())].copy()

In [19]:
# Faz o ajuste das contas do calypso, somando pelas variáveis abaixo e pegando somente as linhas negativas
calypso_ajuste = calypso.groupby(by=['conta_pb','data_ref', 'ticker','currency']).sum().reset_index()
calypso_ajuste = calypso_ajuste[calypso_ajuste['quantidade']<0].copy()
calypso_ajuste['acc'] = calypso_ajuste['conta_pb'].map(dict_contas)
calypso_ajuste = calypso_ajuste.groupby(by=['acc','data_ref', 'ticker','currency']).sum().reset_index()
calypso['acc'] = calypso['conta_pb'].map(dict_contas)

In [20]:
# Aqui ainda faz parte do calypso_ajuste. Estou transferindo as informações entre as bases
calypso = pd.merge(calypso, calypso_ajuste, on=['acc', 'data_ref', 'ticker', 'currency'], how='left')
#Aqui eu estou tirando as linhas que não foram encontradas na volta. lembrar que o Calypso_teste derrubou um monte de linhas
#todas aquelas que não tinham as informações que eram de aluguel doado.
#Assim, teoricamente, todas as linhas desse novo calypso dizem respeito a posições tomadoras de aluguel.
calypso = calypso[calypso['quantidade_y'].isnull()==False].copy().reset_index(drop=True)
calypso = calypso.drop(['quantidade_y', 'year_y'], axis=1).rename(columns={
    'quantidade_x':'quantidade',
    'year_x':'year',
})

In [21]:
def info_base_pb_to_get_ticker(pb, x):
    ticker = str(pb[pb.index==x]['ticker'][x]).replace('/', '')
    isin = str(pb[pb.index==x]['isin'][x])
    cusip = str(pb[pb.index==x]['cusip'][x])
    sedol = str(pb[pb.index==x]['sedol'][x])

    return ticker, isin, cusip, sedol

In [22]:
def get_ticker_bases(calypso, pb, x):
    try:
        ticker_pb, isin, cusip, sedol = info_base_pb_to_get_ticker(pb, x)
        try:
            ticker = calypso[calypso['isin']==isin].drop_duplicates(subset=['ticker']).reset_index(drop=True)['ticker'][0]
            construct = 'isin'
        except IndexError:
            try:
                ticker = calypso[calypso['cusip']==cusip].drop_duplicates(subset=['ticker']).reset_index(drop=True)['ticker'][0]
                construct = 'cusip'
            except IndexError:
                ticker = calypso[calypso['sedol']==sedol].drop_duplicates(subset=['ticker']).reset_index(drop=True)['ticker'][0]
                construct = 'sedol'
        return [ticker, construct]
    except IndexError:
        ticker_pb, isin, cusip, sedol = info_base_pb_to_get_ticker(pb, x)
        print('O ticker %s não foi encontrado pelos códigos.\n Tentaremos agora pelo nome' %(ticker_pb))
        try:
            ticker = ticker_pb.split('.')[0]
            ticker = calypso[calypso.ticker.str.contains(ticker_pb)].drop_duplicates(subset=['ticker'])\
                                                            .reset_index(drop=True)['ticker'][0]
            construct = 'nome'
            print('O ticker %s não foi encontrado pelo nome.\n Ele será então contruído pelo Ticker + Isin' %(ticker_pb))
        except IndexError:
            ticker = ticker.split('.')[0]
            ticker = str(ticker_pb)+' '+str(isin)[:2]
            construct = 'construido_ticker'
            print ('''Nós aconselhamos que você confira o ticker %s, porque ele foi ajustado pelo Isin.
Caso ele esteja ERRADO você deverá fazer o ajuste na próxima parte.''' %(ticker_pb))
        return [ticker, construct]
    except ValueError:
        print(r'O ticker %s não foi encontrado em nenhuma instância.\n Proceder com inserção manual' %(ticker_pb))
        return [np.nan, np.nan]

In [23]:
get_ticker_bases(calypso, pb, 0)

['FMG AU', 'isin']

In [24]:
mapa_tickers = pb[['ticker','isin','cusip','sedol']].drop_duplicates().reset_index(drop=True)

In [25]:
lista_tickers = [get_ticker_bases(calypso, mapa_tickers, x) for x in mapa_tickers.index]

O ticker VALE não foi encontrado pelos códigos.
 Tentaremos agora pelo nome
O ticker VALE não foi encontrado pelo nome.
 Ele será então contruído pelo Ticker + Isin
O ticker BSMXB_OLD.MX não foi encontrado pelos códigos.
 Tentaremos agora pelo nome
Nós aconselhamos que você confira o ticker BSMXB_OLD.MX, porque ele foi ajustado pelo Isin.
Caso ele esteja ERRADO você deverá fazer o ajuste na próxima parte.
O ticker EFGINT não foi encontrado pelos códigos.
 Tentaremos agora pelo nome
Nós aconselhamos que você confira o ticker EFGINT, porque ele foi ajustado pelo Isin.
Caso ele esteja ERRADO você deverá fazer o ajuste na próxima parte.
O ticker GOLD não foi encontrado pelos códigos.
 Tentaremos agora pelo nome
O ticker GOLD não foi encontrado pelo nome.
 Ele será então contruído pelo Ticker + Isin
O ticker FOUREC não foi encontrado pelos códigos.
 Tentaremos agora pelo nome
Nós aconselhamos que você confira o ticker FOUREC, porque ele foi ajustado pelo Isin.
Caso ele esteja ERRADO você de

In [26]:
mapeados = pd.DataFrame(lista_tickers).rename(columns={0:'ticker_calypso', 1:'construct'})

In [27]:
mapa_tickers = pd.concat([mapa_tickers, mapeados], axis=1)

In [28]:
mapa_tickers[mapa_tickers['construct']=='construido_ticker']

,ticker,isin,cusip,sedol,ticker_calypso,construct
99,BSMXB_OLD.MX,NaN,99AJNSZQ9,NaN,BSMXB_OLD.MX na,construido_ticker
109,EFGINT,CH0022268228,99ABKGR43,B0LF188,EFGINT CH,construido_ticker
261,FOUREC,FR0013341955,99GDBHRU3,BG88TW2,FOUREC FR,construido_ticker
339,FECCF,CA35905B1076,35905B107,BYSX6T9,FECCF CA,construido_ticker
356,CNI,CA1363751027,136375102,2210959,CNI CA,construido_ticker
442,ARCO,VGG0457F1071,NaN,B529PQ0,ARCO VG,construido_ticker
459,EVOP,US26927E1047,NaN,BG5NSM3,EVOP US,construido_ticker
480,NRG,US6293775085,NaN,2212922,NRG US,construido_ticker
481,DATA,US87336U1051,NaN,B8DJFL5,DATA US,construido_ticker


In [29]:
ticker_diffs = ticker_diffs.rename(columns={
    'Stock':'ticker',
    'Isin':'isin',
    'Cusip':'cusip',
    'Sedol':'sedol'
})

In [30]:
#mapa_tickers_diffs = ticker_diffs[['ticker','isin','cusip','sedol']].drop_duplicates().reset_index(drop=True)

In [31]:
#lista_tickers_diffs = [get_ticker_bases(calypso, mapa_tickers_diffs, x) for x in mapa_tickers_diffs.index]

In [32]:
#mapeados_diff = pd.DataFrame(lista_tickers_diffs).rename(columns={0:'ticker_calypso', 1:'construct'})

In [33]:
#mapa_tickers_diffs = pd.concat([mapa_tickers_diffs, mapeados_diff], axis=1)

In [34]:
mapa_tickers = mapa_tickers.drop(['isin', 'cusip', 'sedol', 'construct'], axis=1)

In [35]:
pb['id'] = range(0, len(pb))

In [36]:
pb_1 = pd.merge(pb, mapa_tickers, on='ticker', how='left').drop_duplicates()

In [37]:
# Isso aqui tem que ser zero para estar tudo ok!
check_1 = pb_1[~pb_1['id'].isin(pb['id'])].shape[0]+pb[~pb['id'].isin(pb_1['id'])].shape[0]
if check_1 == 0:
    pb = pb_1.drop(['ticker', 'isin','cusip','sedol','id'], axis=1)[['conta', 'pb', 'data_ref','ticker_calypso', 'currency',
                                                                    'qt', 'accrual', 'month','year']].rename(columns={
        'ticker_calypso':'ticker'
    })
else:
    print('Houve algum problema na hora do mapeamento! será necessário fazer ajustes')

# Batimento mensal

#### Bate os arquivos do calypso contra o pb para ver se todos os que estão no calypso estão no pb

In [38]:
calypso_month = calypso.drop(['book', 'isin', 'cusip', 'month', 'year'], axis=1).reset_index(drop=True).copy()
pb_month = pb.drop(['month', 'year'], axis=1).reset_index(drop=True).copy()
pb_month['acc'] = pb_month['conta'].map(dict_contas)

In [39]:
dias_calypso = calypso_month[['acc', 'data_ref']].copy()
dias_calypso['tipo'] = 'calypso'
dias_pb = pb_month[['acc', 'data_ref']].copy()
dias_pb['tipo'] = 'pb'
dias_ok = pd.concat([dias_calypso, dias_pb], axis=0)
dias_ok['check'] = 1
pvt = pd.pivot_table(dias_ok, values=['check'],
                     index='data_ref', columns=['tipo','acc'])

In [40]:
pvt

check                                                
tipo       calypso                         pb                     
acc          int_c ms_cayman ubs_cayman int_c ms_cayman ubs_cayman
data_ref                                                          
2017-12-29     NaN       1.0        1.0   NaN       NaN        NaN
2018-01-01     1.0       1.0        1.0   1.0       1.0        1.0
2018-01-02     1.0       1.0        1.0   1.0       1.0        1.0
2018-01-03     1.0       1.0        1.0   1.0       1.0        1.0
2018-01-04     1.0       1.0        1.0   1.0       1.0        1.0
2018-01-05     1.0       1.0        1.0   1.0       1.0        1.0
2018-01-08     1.0       1.0        1.0   1.0       1.0        1.0
2018-01-09     1.0       1.0        1.0   1.0       1.0        1.0
2018-01-10     1.0       1.0        1.0   1.0       1.0        1.0
2018-01-11     1.0       1.0        1.0   1.0       1.0        1.0
2018-01-12     1.0       1.0        1.0   1.0       1.0        1.0
2018-01-15     1.0       1.0        1.0   1.0       1.0        1.0
2018-01-16     1.0       1.0        1.0   1.0       1.0        1.0
2018-01-17     NaN       NaN        NaN   NaN       1.0        1.0
2018-01-18     1.0       1.0        1.0   1.0       1.0        1.0
2018-01-19     1.0       1.0        1.0   1.0       1.0        1.0
2018-01-22     1.0       1.0        1.0   1.0       1.0        1.0
2018-01-23     1.0       1.0        1.0   1.0       1.0        1.0
2018-01-24     1.0       1.0        1.0   1.0       1.0        1.0
2018-01-25     1.0       1.0        1.0   1.0       1.0        1.0
2018-01-26     1.0       1.0        1.0   1.0       1.0        1.0
2018-01-29     1.0       1.0        1.0   1.0       1.0        1.0
2018-01-30     1.0       1.0        1.0   1.0       1.0        1.0
2018-01-31     1.0       1.0        1.0   1.0       1.0        1.0
2018-02-01     1.0       1.0        1.0   1.0       1.0        1.0
2018-02-02     1.0       1.0        1.0   1.0       1.0        1.0
2018-02-05     1.0       1.0        1.0   1.0       1.0        1.0
2018-02-06     1.0       1.0        1.0   1.0       1.0        1.0
2018-02-07     1.0       1.0        1.0   1.0       1.0        1.0
2018-02-08     1.0       1.0        1.0   1.0       1.0        1.0
...            ...       ...        ...   ...       ...        ...
2018-07-17     1.0       1.0        1.0   1.0       1.0        1.0
2018-07-18     1.0       1.0        1.0   1.0       1.0        1.0
2018-07-19     1.0       1.0        1.0   1.0       1.0        1.0
2018-07-20     1.0       1.0        1.0   1.0       1.0        1.0
2018-07-23     1.0       1.0        1.0   1.0       1.0        1.0
2018-07-24     1.0       1.0        1.0   1.0       1.0        1.0
2018-07-25     1.0       1.0        1.0   1.0       1.0        1.0
2018-07-26     1.0       1.0        1.0   1.0       1.0        1.0
2018-07-27     1.0       1.0        1.0   1.0       1.0        1.0
2018-07-30     1.0       1.0        1.0   1.0       1.0        1.0
2018-07-31     1.0       1.0        1.0   1.0       1.0        1.0
2018-08-01     1.0       1.0        1.0   1.0       1.0        1.0
2018-08-02     1.0       1.0        1.0   1.0       1.0        1.0
2018-08-03     1.0       1.0        1.0   1.0       1.0        1.0
2018-08-06     1.0       1.0        1.0   1.0       1.0        1.0
2018-08-07     1.0       1.0        1.0   1.0       1.0        1.0
2018-08-08     1.0       1.0        1.0   1.0       1.0        1.0
2018-08-09     1.0       1.0        1.0   1.0       1.0        1.0
2018-08-10     1.0       1.0        1.0   1.0       1.0        1.0
2018-08-13     1.0       1.0        1.0   1.0       1.0        1.0
2018-08-14     1.0       1.0        1.0   1.0       1.0        1.0
2018-08-15     1.0       1.0        1.0   1.0       1.0        1.0
2018-08-16     1.0       1.0        1.0   1.0       1.0        1.0
2018-08-17     1.0       1.0        1.0   1.0       1.0        1.0
2018-08-20     1.0       1.0        1.0   1.0       1.0     

In [41]:
#Agrupa as informações que estão no calypso eliminando books. Junta em conta_pb, agent, data_ref, ticker e currency
calypso_batimento = calypso_month.groupby(by=['conta_pb','agent','data_ref','ticker','currency']).sum().copy().reset_index()
calypso_batimento['acc'] = calypso_batimento['conta_pb'].map(dict_contas)
calypso_batimento = calypso_batimento[calypso_batimento['quantidade']<0].reset_index(drop=True)
calypso_batimento = calypso_batimento.groupby(by=['acc','data_ref', 'ticker','currency']).sum().reset_index()  
#é somente um indice para poder checar quais informações do calypso não passaram pelo filtro da PB
calypso_batimento['id'] = range(1, calypso_batimento.shape[0] +1)
pb_month = pb_month.groupby(by=['acc','data_ref','ticker','currency']).sum().reset_index()

In [42]:
calypso_bate_pb = pd.merge(pb_month, calypso_batimento, on=['acc','data_ref','ticker','currency'],  how='left')

In [43]:
check_2 = calypso_bate_pb.shape[0]-pb_month.shape[0]
if check_2!=0:
    print ('%s linhas foram duplicadas durante o merge. Favor verificar se houve algo de errado' %check_2)

In [44]:
# Sobra de destino são os tickers que estavam no pb, mas não tinham correspondência no Calypso
sobra_destino = calypso_bate_pb[(calypso_bate_pb['quantidade'].isnull()==True)&(calypso_bate_pb['accrual']!=0)]

In [45]:
calypso_bate_pb = calypso_bate_pb[calypso_bate_pb['quantidade'].isnull()==False]

In [46]:
# Sobra de origem são os tickers que estavam no Calypso, mas não tinham correspondência no pb
sobra_origem = calypso_batimento[~calypso_batimento['id'].isin(calypso_bate_pb['id'].dropna().drop_duplicates())]

In [47]:
pct_in_pb_not_in_calypso = (1-calypso_bate_pb.shape[0]/pb_month.shape[0])*100
pct_in_calypso_not_in_pb=calypso_batimento[~calypso_batimento['id'].isin(calypso_bate_pb['id'].dropna().drop_duplicates())].shape[0]/\
                                calypso_batimento.shape[0]*100 
texto_1 = '%s pct dos trades estão no prime broker, mas não acharam correspondência no Calypso' % pct_in_pb_not_in_calypso
texto_2 = '%s pct dos trades foram achados no calypso, mas não têm correspondência no prime broker' % pct_in_calypso_not_in_pb

print (texto_1)
print(texto_2)

7.975581587196833 pct dos trades estão no prime broker, mas não acharam correspondência no Calypso
1.3965986635081145 pct dos trades foram achados no calypso, mas não têm correspondência no prime broker


In [48]:
batido = calypso_bate_pb[calypso_bate_pb['qt']==calypso_bate_pb['quantidade']].copy()
batido['status']='batido'
divergentes = calypso_bate_pb[calypso_bate_pb['qt']!=calypso_bate_pb['quantidade']].copy()
corrigidos = divergentes[((divergentes['ticker'].isin(ticker_diffs['stock']))&(divergentes['acc']!='ubs_cayman'))].copy()
corrigidos['status'] = 'corrigido'
n_batidos = divergentes[~((divergentes['ticker'].isin(ticker_diffs['stock']))&(divergentes['acc']!='ubs_cayman'))].copy()
n_batidos['status']='n_batido'
distribuidos = pd.concat([batido, corrigidos, n_batidos], axis=0)

### Distribuição por books

In [49]:
calypso_month_books = calypso.copy()

In [50]:
calypso_month_books = pd.merge(calypso_month_books, calypso_batimento.drop('id', axis=1), \
                               on=['acc','data_ref','ticker','currency'], how='left').rename(columns={
    'quantidade_x':'quantidade_book',
    'quantidade_y':'quantidade_ticker'
})

In [51]:
check_3 = calypso_month_books.shape[0]-calypso.shape[0]
if check_3 != 0:
    print ('%s linhas foram adicionadas durante o merge. Favor verificar quais erros aconteceram.' %check_3)

In [52]:
calypso_month_books = calypso_month_books[calypso_month_books['quantidade_ticker'].isnull()==False]

In [53]:
check_4 = calypso_month_books.shape[0]-calypso.shape[0]
if check_4 != 0 :
    print ('%s trades foram perdidos durante o merge. Favor ferificar quais erros aconteceram.' %check_4)

In [54]:
# Aqui a gente está adicionando a coluna de Accrual pros trades do calyspo
calypso_month_books = pd.merge(calypso_month_books, distribuidos, on=['acc','data_ref','ticker','currency'], how='left')
calypso_month_books = calypso_month_books[calypso_month_books['accrual'].isnull()==False]

In [55]:
check_5 = calypso_month_books.shape[0]-calypso.shape[0]
print ('%s trades não foram encontrados durante a distribuição. Eles provavelmente não foram batidos.' %check_5)

-4061 trades não foram encontrados durante a distribuição. Eles provavelmente não foram batidos.


In [56]:
calypso_month_books = calypso_month_books[calypso_month_books['quantidade_book']!=0]
check_6 = calypso.shape[0]-calypso_month_books.shape[0]
print ('''%s linhas foram retiradas da base por terem quantidade zerada. Ainda restam %s linhas de operações.
Ajustes na base do calypso estão prontos. Dados estão prontos para serem distribuídos entre os books.''' \
                                                                       %(check_6, calypso_month_books.shape[0]))

81924 linhas foram retiradas da base por terem quantidade zerada. Ainda restam 40181 linhas de operações.
Ajustes na base do calypso estão prontos. Dados estão prontos para serem distribuídos entre os books.


In [57]:
por_conta = calypso_month_books.groupby(by=['conta_pb','data_ref','ticker','currency']).sum().reset_index()

In [58]:
# Juntando a quantidade por sub_account do pb para poder dropar as contas positivas (que estão net doadas)
calypso_month_books = pd.merge(calypso_month_books, por_conta.drop(['year', 'qt', 'accrual',\
                                                                   'quantidade', 'id', 'quantidade_ticker'], axis=1), \
                                                       on=['conta_pb','data_ref','ticker','currency'], how='left')\
.rename(columns={
    'quantidade_book_x':'quantidade_book',
    'quantidade_book_y':'quantidade_conta_pb'
})

In [59]:
calypso_month_books = calypso_month_books[calypso_month_books['quantidade_conta_pb']<0]

In [60]:
calypso_month_books = calypso_month_books[calypso_month_books['accrual'].isnull()==False]
print ('Foram feitos os casamentos de trade por sub_account')

Foram feitos os casamentos de trade por sub_account


In [61]:
abertura = calypso_month_books[calypso_month_books['id'].isin(distribuidos['id'].drop_duplicates())==True]
trades_out_abertura = calypso_month_books[calypso_month_books['id'].isin(distribuidos['id'].drop_duplicates())==False]
if trades_out_abertura.shape[0]!=0:
    print('ATENÇãO!!! %s trades foram encontrados em ambas as bases, porém não foram distribuídos.'%trades_out_abertura.shape[0])

In [62]:
abertura = abertura.drop(['isin', 'cusip','month', 'year', 'quantidade', 'id'], axis=1)

In [63]:
abertura['percent'] = abertura['quantidade_book']/abertura['quantidade_ticker']

In [64]:
check_7 = abertura.groupby(by=['acc','data_ref','ticker','currency']).sum().reset_index()
check_7 = check_7[np.isclose(check_7['percent'], 1)==False].shape[0]
if check_7 != 0:
    print ('O batimento não foi feito!!! %s contas somam valor diferente a 100\%')

In [65]:
abertura['accrual_final'] = abertura['percent']*abertura['accrual']

In [66]:
abertura = abertura.drop(['conta_pb', 'quantidade_ticker', 'qt', 'accrual','quantidade_conta_pb', 
                          'percent'], axis=1).reset_index(drop=True)

In [67]:
# Tickers que estão batidos ou que foram corrigidos. Todos distribuídos
abertura_batidos = abertura[abertura['status']!='n_batido']
# Tickers que NÃO ESTÃO BATIDOS EM QUANTIDADE, mas que foram distribuídos, pois foram encontrados em ambas as bases
abertura_n_batidos = abertura[abertura['status']=='n_batido']

In [68]:
bate_soma = abertura.groupby(by=['acc', 'currency']).sum().reset_index()
bate_soma_pb = pb_month.groupby(by=['acc', 'currency']).sum().reset_index()

In [69]:
check_8 = pd.merge(bate_soma_pb, bate_soma, on=['acc', 'currency'], how='left').rename(columns={
'qt':'quantidade_pb',
'accrual':'custos_pb',
'quantidade_book':'quantidade_calypso',
'accrual_final':'custos_batidos'
})
check_8['bate_qt'] = check_8['quantidade_pb']-check_8['quantidade_calypso']
check_8['bate_custos'] = check_8['custos_pb']-check_8['custos_batidos']
check_8['bate_custos_pct'] = check_8['custos_pb']/check_8['custos_batidos']
check_8 = check_8[['acc', 'currency', 'quantidade_pb', 'quantidade_calypso', 'bate_qt', 'custos_pb', 'custos_batidos',
      'bate_custos', 'bate_custos_pct']]   

In [70]:
check_9 = check_8[(check_8['bate_custos_pct']<0.95)|(check_8['bate_custos_pct']>1.05)].shape[0]
if check_9!=0:
    print('%s contas tiveram diferença maior do que 5pct do valor do prime broker' %check_9)

3 contas tiveram diferença maior do que 5pct do valor do prime broker


In [71]:
check_8.drop(['quantidade_pb','quantidade_calypso','bate_qt'],axis=1)

,acc,currency,custos_pb,custos_batidos,bate_custos,bate_custos_pct
0,int_c,AUD,-21091.06,-21091.06,0.000000e+00,1.000000
1,int_c,CHF,-141.22,-141.22,0.000000e+00,1.000000
2,int_c,MXN,-5078.40,-5078.40,9.094947e-13,1.000000
3,int_c,USD,-121603.91,-121603.91,-2.910383e-11,1.000000
4,ms_cayman,CHF,-38.06,NaN,NaN,NaN
5,ms_cayman,EUR,-1199.73,-1198.15,-1.580000e+00,1.001319
6,ms_cayman,GBP,-9.38,-8.00,-1.380000e+00,1.172500
7,ms_cayman,MXN,-7912017.30,-7791583.94,-1.204334e+05,1.015457
8,ms_cayman,SEK,-17.52,-17.52,0.000000e+00,1.000000
9,ms_cayman,USD,-1385614.27,-1318190.00,-6.742427e+04,1.051149


In [67]:
abertura_batidos_ajustado = abertura_batidos.copy()

In [68]:
abertura_batidos_ajustado = abertura_batidos_ajustado.rename(columns={'accrual_final':'accrual'})

In [69]:
abertura_batidos_ajustado_neg = abertura_batidos_ajustado[abertura_batidos_ajustado['accrual']<0]

In [70]:
abertura_batidos_ajustado_posi = abertura_batidos_ajustado[abertura_batidos_ajustado['accrual']>=0]

In [71]:
ajuste_neg_grouped = abertura_batidos_ajustado_neg.groupby(by=['acc','currency']).sum().reset_index()

In [72]:
check_13 = abertura_batidos_ajustado_neg.shape[0]-\
                        pd.merge(abertura_batidos_ajustado_neg, ajuste_neg_grouped, on=['acc','currency'], how='left').shape[0]
if check_13 != 0:
    print ('%s linhas foram duplicadas durante o ajuste dos números negativos. Verificar!!' %check_13)

In [73]:
abertura_batidos_ajustado_neg = pd.merge(abertura_batidos_ajustado_neg, ajuste_neg_grouped, on=['acc','currency'], how='left')

In [74]:
abertura_batidos_ajustado_neg['pct'] = abertura_batidos_ajustado_neg['accrual_x']/abertura_batidos_ajustado_neg['accrual_y']

In [75]:
check_14 = abertura_batidos_ajustado_neg.groupby(by=['acc','currency']).sum().reset_index()
check_14 = check_14[np.isclose(check_14['pct'], 1)==False].shape[0]
if check_14 != 0:
    print ('O batimento não foi feito!!! %s contas somam valor diferente a 100\%')

In [76]:
abertura_batidos_ajustado_posi = abertura_batidos_ajustado_posi.groupby(by=['acc','currency']).sum().reset_index()

In [77]:
check_15 = abertura_batidos_ajustado_neg.shape[0]-\
                                    pd.merge(abertura_batidos_ajustado_neg, abertura_batidos_ajustado_posi, \
                                                         on=['acc','currency'], how='left').shape[0]
if check_15 != 0:
    print ('%s linhas foram duplicadas durante o ajuste de quantidades positivas!!!'%check_15)

In [78]:
abertura_batidos_ajustado_neg = pd.merge(abertura_batidos_ajustado_neg, abertura_batidos_ajustado_posi, \
                                         on=['acc','currency'], how='left').fillna(0)

In [79]:
abertura_batidos_ajustado_neg['distrib_positives'] = abertura_batidos_ajustado_neg['pct']\
                                                        *abertura_batidos_ajustado_neg['accrual']

In [80]:
abertura_batidos_ajustado_neg['accrual_final'] = abertura_batidos_ajustado_neg['accrual_x']+\
                                                        abertura_batidos_ajustado_neg['distrib_positives']

In [81]:
abertura_batidos_ajustado = abertura_batidos_ajustado_neg.drop(['accrual_x', 'accrual_y', 'pct', 'accrual', 
                                                          'distrib_positives'], axis=1)

In [82]:
abertura_batidos_consol = abertura_batidos_ajustado.groupby(['acc', 'book', 'currency'])\
                                        .sum().reset_index().drop(['quantidade_book','quantidade_book_y'], axis=1)\
    .rename(columns={'quantidade_book_x':'quantidade_book'})

In [83]:
data = abertura_batidos_ajustado.data_ref.max()

In [84]:
abertura_batidos_consol_dia = abertura_batidos_ajustado[abertura_batidos_ajustado.data_ref==\
                                                        abertura_batidos_ajustado.data_ref.max()]\
                                                            .groupby(['acc', 'book', 'currency']).sum().reset_index()\
                                                                .drop(['quantidade_book','quantidade_book_y'], axis=1)\
                                                                    .rename(columns={'quantidade_book_x':'quantidade_book'})

### Abertura por WM

In [85]:
pb_wm = abertura[(abertura['book'].str.contains('PRIVATE'))\
                 |(abertura['book'].str.contains('CLIENT'))].copy()

In [86]:
calypso_wm = bdwm.copy()
calypso_wm['tipo'] = 'WM' 
calypso_wm['date'] = pd.to_datetime(calypso_wm['date'])
calypso_wm['month_number'] = calypso_wm['date'].dt.month
calypso_wm['year'] = calypso_wm['date'].dt.year
calypso_wm['month'] = calypso_wm['month_number'].map(months_dict)
calypso_wm = calypso_wm[['Account Name','tipo', 'Agent','date', 'shares', 'Book', 'PRODUCT_CODE.TICKER', 'PRODUCT_CODE.ISIN', 
                   'PRODUCT_CODE.CUSIP', 'Product Currency', 'month', 'year', 'LE Name']].reset_index(drop=True)
calypso_wm.columns = ['conta_pb','conta', 'agent', 'data_ref', 'quantidade', 'book', 'ticker', 'isin', 'cusip', 
              'currency', 'month', 'year', 'cliente']
calypso_wm['quantidade'] = calypso_wm['quantidade'].replace(',','', regex=True).astype(float)

In [87]:
calypso_wm['quantidade'] = -1*calypso_wm['quantidade']

In [88]:
calypso_wm = calypso_wm[calypso_wm['quantidade']<0]

In [89]:
calypso_wm_grouped =calypso_wm.groupby(by=['data_ref', 'book', 'ticker', 'currency']).sum().reset_index()

In [90]:
check_17 = pb_wm.shape[0]-pd.merge(pb_wm, calypso_wm_grouped, on=['data_ref', 'book', 'ticker', 'currency'], how='left')\
                                                                                    .shape[0]
if check_17!=0:
    print ('%s linhas foram duplicadas durante o barimento de WM!!!! Verificar!' %check_17)

In [91]:
batimento_wm = pd.merge(pb_wm, calypso_wm_grouped, on=['data_ref', 'book', 'ticker', 'currency'], how='left')\
                                                                                    .drop('year', axis=1)

In [92]:
batimento_wm_sobra_destino = batimento_wm[batimento_wm['quantidade'].isnull()==True].copy()
print ('Durante o batimento de WM, %s trades ficaram como sobra de destino!!' %batimento_wm_sobra_destino.shape[0])

batimento_wm_sobra_destino['status'] = 'n_batido'

Durante o batimento de WM, 0 trades ficaram como sobra de destino!!


In [93]:
batimento_wm = batimento_wm[batimento_wm['quantidade'].isnull()==False].copy()
batimento_wm_batidos = batimento_wm[batimento_wm['quantidade_book']==batimento_wm['quantidade']].copy()

batimento_wm_batidos['status'] = 'batido'

In [94]:
batimento_wm_divergentes = batimento_wm[batimento_wm['quantidade_book']!=batimento_wm['quantidade']].copy()
print ('Durante o batimento de WM, %s trades ficaram com quantidade divergente!!' %batimento_wm_divergentes.shape[0])

Durante o batimento de WM, 0 trades ficaram com quantidade divergente!!


In [95]:
batimento_wm_divergentes['status'] = 'divergente'

In [96]:
batimento_wm = pd.concat([batimento_wm_batidos, batimento_wm_divergentes], axis=0)

In [97]:
batimento_wm['id'] = range(0, batimento_wm.shape[0])

In [98]:
batimento_wm_distrib = pd.merge(calypso_wm, batimento_wm, on=['data_ref', 'book', 'ticker', 'currency'], how='left')\
[['conta_pb_x', 'agent_x', 'data_ref', 'quantidade_x', 'book', 'ticker', 'currency', 'cliente','quantidade_book', 
  'acc', 'status','accrual_final', 'id']].rename(columns={
    'quantidade_book':'quantidade_ticker',
    'quantidade_x':'quantidade_book',
    'conta_pb_x':'conta_pb',
    'agent_x':'agent',
    'cliente':'conta'   
})
check_18 = batimento_wm_distrib.shape[0]-calypso_wm.shape[0]
if check_18!=0:
    print ('%s linhas foram duplicadas durante o ajuste de WM' %check_18)

In [99]:
batimento_wm_sobra_origem = batimento_wm[~batimento_wm_distrib['id'].isin(batimento_wm['id'].copy().drop_duplicates())].copy()
print ('Durante o batimento de WM, %s trades não foram distribuídos!!' %batimento_wm_sobra_origem.shape[0])

Durante o batimento de WM, 55 trades não foram distribuídos!!


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [100]:
batimento_wm_distrib = batimento_wm_distrib.drop('id', axis=1)

In [101]:
n_batimento_wm_distrib = batimento_wm_distrib[batimento_wm_distrib['quantidade_ticker'].isnull()==True]\
                                                                .drop(['quantidade_ticker'], axis=1)

In [102]:
batimento_wm_distrib = batimento_wm_distrib[batimento_wm_distrib['quantidade_ticker'].isnull()==False]

In [103]:
batimento_wm_distrib['pct'] = batimento_wm_distrib['quantidade_book']/batimento_wm_distrib['quantidade_ticker']

In [104]:
batimento_wm_distrib.groupby(by=['data_ref', 'book', 'ticker', 'currency']).sum()

conta_pb  quantidade_book  \
data_ref   book                ticker  currency                              
2018-08-01 BD_CLIENTS_UBS      BAP US  USD        1823956          -7500.0   
                               ENPH US USD         214962         -80000.0   
                               LTM US  USD        1823956         -55193.0   
                               MELI US USD        1823956          -5364.0   
                               PAGS US USD        1823956          -1000.0   
           PRIVATE_DEFAULT_UBS BOX US  USD         463312          -2000.0   
                               CAKE US USD        1682709          -1000.0   
                               DRI US  USD        1682709           -600.0   
                               LE US   USD        1682709          -1500.0   
                               MELI US USD        1682709           -150.0   
                               NFLX US USD          46537          -1000.0   
                               PAGS US USD          46537         -16000.0   
                               WU US   USD        1682709          -2500.0   
2018-08-02 BD_CLIENTS_UBS      BAP US  USD        1823956          -7500.0   
                               ENPH US USD         214962         -80000.0   
                               LTM US  USD        1823956         -55193.0   
                               MELI US USD        1823956          -5364.0   
                               PAGS US USD        1823956          -1000.0   
           PRIVATE_DEFAULT_UBS BOX US  USD         463312          -2000.0   
                               CAKE US USD        1682709          -1000.0   
                               DRI US  USD        1682709           -600.0   
                               LE US   USD        1682709          -1500.0   
                               MELI US USD        1682709           -150.0   
                               NFLX US USD          46537          -1000.0   
                               PAGS US USD          46537         -16000.0   
                               WU US   USD        1682709          -2500.0   
2018-08-03 BD_CLIENTS_UBS      BAP US  USD        1823956          -7500.0   
                               ENPH US USD         214962         -63208.0   
                               LTM US  USD        1823956         -55193.0   
                               MELI US USD        1823956          -5364.0   
...                                                   ...              ...   
2018-08-10 BD_CLIENTS_UBS      BSAC US USD         424314          -9002.0   
                               ENIA US USD         424314          -6600.0   
                               ENPH US USD         214962         -63208.0   
                               LTM US  USD        1823956         -55193.0   
                               MELI US USD        1823956          -5364.0   
                               PAGS US USD        1823956          -1000.0   
           PRIVATE_DEFAULT_UBS CAKE US USD        1682709           -700.0   
                               DRI US  USD        1682709           -600.0   
                               KO US   USD        1682709           -700.0   
                               LE US   USD        1682709          -1500.0   
                               MELI US USD        1682709           -150.0   
                               NFLX US USD          46537          -1000.0   
                               PAGS US USD          46537         -16000.0   
                               TSLA US USD        1682709           -200.0   
                               WU US   USD        1682709          -2500.0   
2018-08-13 BD_CLIENTS_UBS      BAP US  USD        1823956          -7500.0   
                               BSAC US USD         424314          -2500.0   
                               ENPH US USD         214962         -63208.0   
                               LTM US  USD        1823956        -10019

In [105]:
batimento_wm_distrib['accrual'] = batimento_wm_distrib['accrual_final']*batimento_wm_distrib['pct']

In [106]:
batimento_wm_distrib = batimento_wm_distrib.drop(['quantidade_ticker', 'pct'], axis=1).rename(columns = {
'accrual_final':'accrual_n_distrib'})

In [107]:
abertura_batidos_consol_wm = batimento_wm_distrib.copy()

In [108]:
abertura_batidos_consol_wm = abertura_batidos_consol_wm[abertura_batidos_consol_wm['accrual']!=0]

In [109]:
'''total_wm = abertura_batidos_consol_wm.groupby(by=['acc','book','currency']).sum().reset_index()

total_wm_calypso = abertura_batidos_consol[(abertura_batidos_consol['book'].str.contains('CLIENTS'))|\
                       (abertura_batidos_consol['book'].str.contains('PRIVATE'))].reset_index(drop=True)\
                                                                .groupby(by=['book', 'currency']).sum().reset_index()

total_wm_n_distrib = pd.merge(total_wm, total_wm_calypso, on=['book','currency'], how='left').fillna(0)

abertura_batidos_consol_wm



total_wm_n_distrib['accrual_final_n_distrib'] = total_wm_n_distrib['accrual_final']-total_wm_n_distrib['accrual']

total_wm_n_distrib = total_wm_n_distrib.drop(['accrual', 'accrual_final', 'quantidade_book_x','quantidade_book_y',
                                              'conta_pb'], axis=1)

abertura_batidos_consol_wm = abertura_batidos_consol_wm.groupby(by=['book','conta','currency']).sum().reset_index()\
                                                .drop(['conta_pb','quantidade_book'], axis=1)

total_wm_n_distrib['conta'] = 'OUTROS'
total_wm_n_distrib = total_wm_n_distrib.rename(columns={'accrual_final_n_distrib':'accrual'})

abertura_batidos_consol_wm = abertura_batidos_consol_wm.groupby(by=['conta', 'book','currency']).sum().reset_index()
abertura_batidos_consol_wm['acc'] = 'ubs_cayman'

abertura_batidos_consol_wm = pd.concat([total_wm_n_distrib, abertura_batidos_consol_wm], axis=0)

abertura_batidos_consol_wm'''

"total_wm = abertura_batidos_consol_wm.groupby(by=['acc','book','currency']).sum().reset_index()\n\ntotal_wm_calypso = abertura_batidos_consol[(abertura_batidos_consol['book'].str.contains('CLIENTS'))|                       (abertura_batidos_consol['book'].str.contains('PRIVATE'))].reset_index(drop=True)                                                                .groupby(by=['book', 'currency']).sum().reset_index()\n\ntotal_wm_n_distrib = pd.merge(total_wm, total_wm_calypso, on=['book','currency'], how='left').fillna(0)\n\nabertura_batidos_consol_wm\n\n\n\ntotal_wm_n_distrib['accrual_final_n_distrib'] = total_wm_n_distrib['accrual_final']-total_wm_n_distrib['accrual']\n\ntotal_wm_n_distrib = total_wm_n_distrib.drop(['accrual', 'accrual_final', 'quantidade_book_x','quantidade_book_y',\n                                              'conta_pb'], axis=1)\n\nabertura_batidos_consol_wm = abertura_batidos_consol_wm.groupby(by=['book','conta','currency']).sum().reset_index()               

In [110]:
abertura_batidos_consol_wm = abertura_batidos_consol_wm[['acc','conta','book','currency','accrual']]

In [111]:
abertura_batidos_consol = abertura_batidos_consol[['acc', 'book', 'currency', 'accrual_final']]

In [112]:
abertura_batidos_consol_dia = abertura_batidos_consol_dia.drop('quantidade_book', axis=1)

# Resultados

In [113]:
string = str(data.year)+'-'+str("{:02}".format(data.month))+'-'+str("{:02}".format(data.day))

In [114]:
save_path = r'\\DRIOC0231PFS\Apoio_SB\Geral\BD\Aluguel_Offshore\Arquivos diários'

In [115]:
subfolder = months_dict[pd.to_datetime(data).month]+str(pd.to_datetime(data).year)
final_path = save_path+'\\'+subfolder+'\\'+string
if os.path.isdir(save_path+'\\'+subfolder)==True:
    if os.path.isdir(save_path+'\\'+subfolder+'\\'+string)==False:
        os.mkdir(save_path+'\\'+subfolder+'\\'+string)
else:
    os.mkdir(save_path+'\\'+subfolder)
    os.mkdir(save_path+'\\'+subfolder+'\\'+string)

In [116]:
final_path

'\\\\DRIOC0231PFS\\Apoio_SB\\Geral\\BD\\Aluguel_Offshore\\Arquivos diários\\ago2018\\2018-08-13'

In [117]:
writer = pd.ExcelWriter(final_path+'\\batimento_alguel_offshore_'+string+'.xlsx')

In [118]:
abertura_batidos_consol.to_excel(writer, sheet_name = 'Custos Consolidados', index=False)

In [119]:
abertura_batidos_consol_wm.to_excel(writer, sheet_name = 'Distrib WM', index=False)

In [120]:
abertura_batidos_consol_dia.to_excel(writer, sheet_name = string, index=False)

In [121]:
writer.save()

In [122]:
writer_2 = pd.ExcelWriter(final_path+'\\batimento_alguel_offshore_sobras_'+string+'.xlsx')

In [123]:
abertura_n_batidos.to_excel(writer_2, sheet_name = 'abertura_n_batidos', index=False)
sobra_origem.to_excel(writer_2, sheet_name = 'sobra_origem', index=False)
#sobra_destino.to_excel(writer_2, sheet_name = 'sobra_destino', index=False)

In [124]:
writer_2.save()

In [125]:
lista_files = [final_path+'\\batimento_alguel_offshore_'+string+'.xlsx',
               final_path+'\\batimento_alguel_offshore_sobras_'+string+'.xlsx']

In [126]:
email_sender('guilherme.correia@btgpactual.com', 'OL-equity-backoffice@btgpactual.com;OL-EQUITYBD@btgpactual.com;Analuisa.Godoy@btgpactual.com', \
             'Aluguel Offshore | Batimento Parcial - %s' %(string), \
             'Legenda: abertura_n_batidos são os que possuem diferença de quantidade. sobra_origem são os que tem no calypso e não no pb', \
             lista_files)